The goal of this notebook is to take the ad hoc implementation in `MIPExample1.ipynb` and vectorize the operations such that any size Tyche problem can be converted to a MIP and solved.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

In [3]:
import cProfile
import timeit

In [4]:
from mip import Model, minimize, BINARY, xsum

In [5]:
designs = ty.Designs("data")
investments = ty.Investments("data")
designs.compile()
tranche_results = investments.evaluate_tranches(designs, sample_count=250)
results = investments.tranches.join(tranche_results.summary)
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Get the wide-format interpolated elicitation data from the Tyche Evaluator and reset the multi-level index.

In [7]:
wide = evaluator.evaluate_corners_wide().reset_index()

**Input needed**: List of investment categories

**Input needed**: List of metrics to optimize and/or constrain

In [10]:
categories = ['Soft Costs']

metrics = ['Capital']

Instantiate the MIP optimization problem.

In [12]:
example = Model()

Fill in the various index values for the one-by-one data set.

In [13]:
# Number of investment categories
Inv = len(categories)

# Number of metrics
J = len(metrics)

# Number of elicited funding levels
l_1 = len(wide_dat)

# Number of linear intervals
n_1 = l_1 - 1

# Number of new lambda variables
k_1 = l_1

Pull out the investment values and metric values from the data set.

In [14]:
# Investment values
v_1 = wide_dat.loc[:,'Soft Costs'].values

# Elicited metric values
m_1 = wide_dat.loc[:, 'Capital'].values

Define upper bound for budget constraint.

In [15]:
B = 3000000.0

Create binary (integer) variables $y_{in_i}$.

In [16]:
y_1 = [example.add_var(name='y_1', var_type=BINARY) for n_i in range(n_1)]

Create continuous $\lambda$ variables with lower bound 0.0 and upper bound 1.0

In [17]:
lmbd_1 = [example.add_var(name='lmbd_1', lb=0.0, ub=1.0) for k_i in range(k_1)]

Create budget constraint as a function of the $\lambda$ variables and the elicited investment levels from the data set.

In [18]:
example += xsum(lmbd_1[i] * v_1[i] for i in range(k_1)) <= B

Convexity constraints on $\lambda$ variables.

In [19]:
example += sum(lmbd_1) == 1

Constrain binary $y$ variables such that at most one of the $y$ variables can be equal to 1.

In [20]:
example += sum(y_1) == 1

Interval constraints on $y$ variables and $\lambda$ variables.

In [21]:
example += y_1[0] <= lmbd_1[0] + lmbd_1[1]

In [22]:
example += y_1[1] <= lmbd_1[1] + lmbd_1[2]

Create objective function: Capital (metric) as a function of $\lambda$s and $y$s.

In [23]:
example.objective = minimize(-1.0 * xsum(lmbd_1[i] * m_1[i] for i in range(k_1)))

Optimize.

In [24]:
example.optimize()

<OptimizationStatus.OPTIMAL: 0>

Print optimal objective function value, with a negative applied to reverse the -1.0 in the objective function definition (minimizing a negative -> maximize a positive).

In [25]:
-1.0 * example.objective_value

-1.3555357403466024

Get the optimal $\lambda$ and $y$ values.

In [26]:
for v in example.vars:
    print('{} : {}'.format(v.name, v.x))

y_1 : 0.0
y_1 : 1.0
lmbd_1 : 0.0
lmbd_1 : 0.5
lmbd_1 : 0.5


\$3,000,000 is halfway between the second and third investment levels, so this solution is the correct optimum.